# Recomendações básicas
Esse notebook almeja desenvolver um sistema de recomendação simples para disciplinas optativas.

## Processamento inicial dos dados

In [37]:
import pandas as pd

dados_formados_path = 'input/dados_formados.csv'
dados_formados = pd.read_csv(dados_formados_path, delimiter=';')
dados_formados.columns

Index(['ID_ANONIMO', 'CR', 'PERIODO', 'INGRESSO', 'CODIGO', 'ANO', 'PERIODO.1',
       'NOTA', 'CH', 'SITUACAOALUNO'],
      dtype='object')

In [38]:
dados_regulares_path = 'input/dados_regulares.csv'
dados_regulares = pd.read_csv(dados_regulares_path, delimiter=';')
dados_regulares.columns

Index(['ID_ANONIMO', 'CR', 'PERIODO', 'INGRESSO', 'CODIGO', 'ANO', 'PERIODO.1',
       'NOTA', 'CH', 'SITUACAOALUNO'],
      dtype='object')

In [39]:
def select_random_student(data : pd.DataFrame, max_tries : int, minimum_entry_year : int):
    id = 0
    tries = 0
    while id == 0 and tries < max_tries:
        try:
            id = data[[int(x[3].split('/')[2].split(' ')[0]) >= 14 for x in data.values]].sample()
            return int(id.ID_ANONIMO.iloc[0])
        except:
            id = 0
            tries += 1
    return None

In [59]:
print(str(select_random_student(dados_formados, 100, 14)))

172046


In [41]:
print(str(select_random_student(dados_regulares, 100, 11)))

232732


In [42]:
dados_regulares[[int(x[3].split('/')[2].split(' ')[0]) >= 14 for x in dados_regulares.values]]

,ID_ANONIMO,CR,PERIODO,INGRESSO,CODIGO,ANO,PERIODO.1,NOTA,CH,SITUACAOALUNO
0,11357,"0,6967",5,23/01/23,CSD21,2023,1,"4,7",45,Regular
1,11357,"0,6967",5,23/01/23,ES70P,2023,1,"7,6",45,Regular
2,11357,"0,6967",5,23/01/23,CSE20,2023,1,6,60,Regular
3,11357,"0,6967",5,23/01/23,ES70G,2023,1,"7,3",45,Regular
4,11357,"0,6967",5,23/01/23,CSF20,2023,1,"6,3",45,Regular
...,...,...,...,...,...,...,...,...,...,...
19877,261036,"0,695",2,09/08/23,ICSF13,2023,2,"6,6",90,Regular
19878,261911,"0,7143",1,17/08/23,ICSD21,2023,2,8,45,Regular
19879,261911,"0,7143",1,17/08/23,ELEX10,2023,2,8,45,Regular
19880,261911,"0,7143",1,17/08/23,ICSF13,2023,2,6,90,Regular


In [43]:
def get_all_data_from_student(student_id : int, data : pd.DataFrame):
    return data[[x == student_id for x in data.ID_ANONIMO]]
    pass

In [45]:
get_all_data_from_student(select_random_student(dados_formados, 100, 14), dados_formados)

,ID_ANONIMO,CR,PERIODO,INGRESSO,CODIGO,ANO,PERIODO.1,NOTA,CH,SITUACAOALUNO
15577,123736,"0,8296",10,24/09/14,FI71M,2014,2,6,60,Formado
15578,123736,"0,8296",10,24/09/14,FI70D,2016,2,"7,8",30,Formado
15579,123736,"0,8296",10,24/09/14,EL62O,2015,1,9,45,Formado
15580,123736,"0,8296",10,24/09/14,MA70H,2015,2,"9,7",60,Formado
15581,123736,"0,8296",10,24/09/14,MA73A,2015,2,"6,8",60,Formado
...,...,...,...,...,...,...,...,...,...,...
15688,123736,"0,8296",10,24/09/14,FI72N,2015,1,"8,2",30,Formado
15689,123736,"0,8296",10,24/09/14,MA70C,2018,2,"8,6",60,Formado
15690,123736,"0,8296",10,24/09/14,QB70E,2018,1,"8,1",30,Formado
15691,123736,"0,8296",10,24/09/14,QB70C,2017,1,"7,6",75,Formado


## Preparo dos dados

Queremos recomendar disciplinas para estudantes com base em sua performance nas disciplinas prévias. Para tal, precisamos converter os dados do DataFrame para um vetor de _features_. Escolhemos para esse teste simples utilizar a nota do estudante como valor a ser analisado. A nota utilizada será a menor entre todas as vezes que a disciplina foi cursada.

| Curso | Nota | Ultima data ((Ano - 2000)*2 + periodo)|
| -- | -- | -- |
| FI73A | 6 |  20 (2009 - 2o periodo) |
| EL68A | -1 ( nao fez ) |  -1 ( nao fez ) |
| GE60C | 4 (passou, mas reprovou uma vez com 4) |  20 (2009 - 2o periodo) |

In [46]:
from typing import List

def get_user_array(student_id : int, data : pd.DataFrame):
    student_data = get_all_data_from_student(student_id, data)
    student_array : List[ List[string], List[float], List[int] ] = [[],[],[]]
    for course in student_data.values:
        if [course[4]] not in student_array[0]:
            print([course[4], float(str(course[7]).replace(',','.')),(course[5] - 2000)*2 + course[6]])
            student_array[0].append(course[4])
            student_array[1].append(float(str(course[7]).replace(',','.')))
            student_array[2].append((course[5] - 2000)*2 + course[6])
        else:
            index = student_array[0].index(course[4])
            nota = float(str(course[7]).replace(',','.'))
            student_array[1] = nota if nota < student_array[1][index] else student_array[1][index]
    
    pass

In [47]:
print(get_user_array(93846, dados_formados))

['EL66C', 7.3, 20]
['IF6AE', 10.0, 25]
['IF6AG', 9.2, 24]
['IF6AL', 9.0, 25]
['IF6AB', 8.9, 25]
['IF60A', 8.8, 24]
['ES60F', 10.0, 19]
['ES60G', 7.8, 20]
['IF69D', 6.1, 24]
['ES60A', 10.0, 18]
['EL62A', 10.0, 17]
['MA63B', 9.9, 18]
['MA61A', 7.0, 16]
['MA63A', 5.2, 18]
['MA65A', 8.7, 18]
['MA62A', 7.5, 17]
['IF60K', 0.0, 25]
['GE60B', 8.1, 22]
['IF61C', 9.3, 16]
['FI62A', 7.3, 17]
['FI66A', 8.1, 20]
['ES65A', 9.1, 24]
['MA61B', 7.2, 16]
['FI61A', 8.9, 16]
['IF63F', 8.3, 18]
['ES61A', 8.6, 16]
['QB62A', 7.7, 17]
['IF62J', 9.0, 17]
['IF66B', 7.8, 21]
['FI63A', 7.6, 17]
['EL68F', 8.2, 23]
['ENADE C', nan, 26]
['GE60D', 6.5, 21]
['EL65A', 7.8, 19]
['FI64C', 7.5, 19]
['IF64C', 5.0, 19]
['EL63B', 5.7, 18]
['IF63C', 8.4, 18]
['IF63E', 6.9, 18]
['ENADE I', 0.0, 18]
['ES60B', 10.0, 24]
['ES60D', 6.8, 25]
['IF66C', 8.4, 21]
['IF66D', 7.0, 21]
['IF65E', 8.6, 20]
['EL65H', 5.0, 20]
['IF65D', 10.0, 20]
['IF64J', 9.6, 19]
['IF65C', 6.3, 20]
['EL64H', 7.6, 20]
['EL64H', 4.0, 19]
['EL65G', 6.0, 19]
['

## Recomendações "ingênuas"

### Recomendando os mais populares

In [48]:
import numpy as np

disciplinas = []
with open('input/dependencias.txt') as f:
    for line in f:
        curr = line.split(';')
        if 'P8' in curr:
            curr = curr[1].split('\n')[0]
            disciplinas.append(curr)

disciplinas_obrigatorias = ['GE70D', 'EEC31', 'CSS30','EEX23']
disciplinas = np.array(disciplinas)
disciplinas = disciplinas[[disc not in disciplinas_obrigatorias for disc in disciplinas]]

dados_formados_path = 'input/dados_formados.csv'
dados_formados = pd.read_csv(dados_formados_path, delimiter=';')

dados_formados_optativas = dados_formados[[int(x[3].split('/')[2].split(' ')[0]) >= 14 for x in dados_formados.values]]
dados_formados_optativas = dados_formados_optativas[[float(str(x[7]).replace(',','.')) >= 6 for x in dados_formados_optativas.values]]
dados_formados_optativas = dados_formados_optativas[[x[4] != 'ES70N' or x[5] > 2017 for x in dados_formados_optativas.values]]
dados_formados_optativas = dados_formados_optativas[[x[4] != 'FI70D' or x[5] > 2017 for x in dados_formados_optativas.values]]
dados_formados_optativas = dados_formados_optativas[[x[4] != 'FI70A' or x[5] > 2017 for x in dados_formados_optativas.values]]
dados_formados_optativas = dados_formados_optativas[[x[4] != 'GE70F' or x[5] > 2017 for x in dados_formados_optativas.values]]
dados_formados_optativas = dados_formados_optativas[dados_formados_optativas.CODIGO.isin(disciplinas)]

In [49]:
from collections import Counter

popular_courses = Counter([materia for materia in dados_formados_optativas["CODIGO"]])

print(popular_courses)

Counter({'CSH30': 61, 'CSV30': 35, 'CSH42': 33, 'CSI53': 18, 'CSV40': 15, 'CSR41': 14, 'CSR44': 14, 'CSR42': 13, 'MA70C': 12, 'ED70T': 11, 'CSB51': 11, 'CSV45': 10, 'CSI41': 10, 'CSB41': 10, 'DI84D': 10, 'CSM41': 9, 'CSM43': 9, 'CSR43': 9, 'CSM44': 9, 'ES70J': 8, 'ES70B': 8, 'CSM40': 7, 'CSB53': 7, 'CSA44': 6, 'ES70N': 6, 'GE70F': 6, 'CSH44': 5, 'CSB54': 5, 'CSH43': 4, 'CSM30': 4, 'CSE40': 4, 'EEY41': 4, 'CSI56': 4, 'ED70U': 4, 'CSA45': 3, 'CSA42': 3, 'FI70D': 3, 'FI70B': 3, 'EL64B': 2, 'CSI58': 2, 'CSR53': 2, 'CSD41': 2, 'CSD40': 2, 'CSG42': 2, 'CSV52': 2, 'CSB52': 2, 'FI70A': 2, 'EL6AE': 1, 'CSI54': 1, 'CSW47': 1, 'CSD52': 1, 'CSR45': 1, 'EEY42': 1, 'CSH41': 1, 'EEY51': 1, 'EEL51': 1, 'CSR48': 1, 'CSI31': 1, 'IF6BV': 1, 'EL6CB': 1, 'CSG44': 1, 'FI70E': 1, 'CSA43': 1, 'EEY43': 1, 'CSG48': 1, 'EEC41': 1, 'CSH45': 1, 'CSI55': 1, 'CSE43': 1, 'CSW45': 1, 'CSI51': 1, 'CSV41': 1, 'CSI57': 1, 'FCH7HB': 1, 'CSR47': 1, 'EEY44': 1, 'CSA41': 1, 'EL75H': 1})


In [119]:
from typing import List

# Recebe um estudante e sugere as disciplinas mais populares que ele não fez ainda
def sugestoes_populares(student_id : int, max_sugestoes : int, data : pd.DataFrame) -> List:
    suggestions = [interest for interest, _ in popular_courses.most_common() if interest not in get_all_data_from_student(student_id, data)["CODIGO"].unique()]
    return suggestions[:max_sugestoes]

In [124]:
aluno = (select_random_student(dados_formados, 100, 14))
print(aluno)

123688


In [125]:
print(sugestoes_populares(aluno, 5, dados_formados))

['CSH42', 'CSI53', 'CSR44', 'MA70C', 'ED70T']


In [127]:
[x for x in get_all_data_from_student(aluno, dados_formados)["CODIGO"] if x in dados_formados_optativas["CODIGO"].unique()]

['CSA44',
 'CSA45',
 'CSH30',
 'CSB41',
 'CSI56',
 'CSR42',
 'CSA42',
 'CSI41',
 'CSR53',
 'CSR41',
 'CSV40',
 'CSV30']

In [128]:
dados_formados_optativas["CODIGO"].unique()

array(['CSM41', 'CSH43', 'EL64B', 'MA70C', 'CSM43', 'CSH30', 'EL6AE',
       'CSI58', 'CSR53', 'CSR41', 'CSI54', 'CSW47', 'CSV30', 'CSH42',
       'CSM40', 'CSD41', 'CSV45', 'CSM30', 'CSE40', 'EEY41', 'CSR43',
       'CSV40', 'CSR44', 'CSI53', 'CSI41', 'CSA44', 'CSA45', 'CSB41',
       'CSI56', 'CSD52', 'CSR45', 'CSD40', 'ES70J', 'ES70B', 'EEY42',
       'DI84D', 'CSM44', 'CSH41', 'ED70T', 'CSR42', 'CSG42', 'ED70U',
       'CSA42', 'CSB51', 'EEY51', 'EEL51', 'CSB53', 'CSR48', 'ES70N',
       'CSH44', 'CSB54', 'CSI31', 'IF6BV', 'GE70F', 'FI70D', 'EL6CB',
       'CSG44', 'FI70E', 'FI70B', 'CSA43', 'EEY43', 'CSG48', 'EEC41',
       'CSH45', 'CSV52', 'CSB52', 'CSI55', 'CSE43', 'CSW45', 'CSI51',
       'CSV41', 'CSI57', 'FI70A', 'FCH7HB', 'CSR47', 'EEY44', 'CSA41',
       'EL75H'], dtype=object)